In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, KFold, cross_val_predict
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import accuracy_score, f1_score, precision_recall_fscore_support


In [2]:
    # 1. 데이터 불러오기
data = pd.read_csv('azdigar nav sartirovka+0 delete.csv')

In [3]:
columns = [
    ('w08chronic_a', '고혈압'), 
    ('w08chronic_b', '당뇨병'),
    ('w08chronic_c', '암 및 악성종양'),
    ('w08chronic_d', '만성 폐질환'),
    ('w08chronic_e', '간질환'),
    ('w08chronic_f', '심장질환'),
    ('w08chronic_g', '뇌혈관질환'),
    ('w08chronic_h', '정신과적 질환'),
    ('w08chronic_i', '관절염 또는 류마티스'),
    ('w08chronic_k', '소화기계 질환'),
    ('w08chronic_l', '디스크 진단'),
    ('w08chronic_m', '지매')

]


In [4]:
# Target 컬럼명만 추출
target_columns = [col[0] for col in columns]

In [5]:
# 3. 데이터 클리닝 - 'w08chronic_m'에서 값이 3인 행 제거
cleaned_data = data[data['w08chronic_m'] != 3]


In [6]:
# # 4. 인코딩 처리 (1 -> 0 (no), 5 -> 1 (yes))
# cleaned_data[target_columns] = cleaned_data[target_columns].replace({1: 0, 5: 1})


In [7]:
# 5. 피처와 타겟 정의
X = cleaned_data.drop(columns=target_columns)
y_all = cleaned_data[target_columns]

In [8]:
# 6. 데이터 스케일링
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)


In [9]:
y = np.array(y_all.values)
y_binarized = np.where(y==5, 1, 0)

In [10]:
# # 7. KFold 교차 검증 설정
# kf = KFold(n_splits=5, shuffle=True, random_state=42)

# 6. 결과 저장을 위한 빈 데이터프레임 생성
result_table = pd.DataFrame(columns=[
    'Target', 'Model', 'Precision', 'Recall', 'F1-Score',
    'Accuracy'
])

# 전체 타겟에 대한 예측 결과를 저장할 리스트를 미리 선언
all_y_test = []
all_y_pred = []

# 7. 각 타겟 열을 사용한 학습 및 평가

# 데이터 나누기
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_binarized, test_size=0.2, random_state=42)



In [11]:
# # 8. 모델 정의
# random_forest = MultiOutputClassifier(RandomForestClassifier(random_state=42))
# decision_tree = MultiOutputClassifier(DecisionTreeClassifier(random_state=42))

# 모델 정의
rf = RandomForestClassifier(n_estimators=100, random_state=42)
multi_target_rf = MultiOutputClassifier(rf, n_jobs=-1)



 # 모델 학습
multi_target_rf.fit(X_train, y_train)

# 예측
y_pred = multi_target_rf.predict(X_test)

for idx, target_column in enumerate(columns):
    # 각 열(클래스)에 대해 예측 값과 실제 값을 비교하여 성능 측정
    accuracy = accuracy_score(y_test[:, idx], y_pred[:, idx])
    precision, recall, f1, _ = precision_recall_fscore_support(y_test[:, idx], y_pred[:, idx], average='binary')

    new_row = pd.DataFrame({
        'Model': ['Random Forest'],
        'Target': [target_column],
        'Precision': [precision],
        'Recall': [recall],
        'F1-Score': [f1],
        'Accuracy': [accuracy],
    })
    result_table = pd.concat([result_table, new_row], ignore_index=True)

# 결과 테이블 출력
print("\n전체 모델 평가 결과:")
print(result_table)



전체 모델 평가 결과:
                         Target          Model  Precision    Recall  F1-Score  \
0           (w08chronic_a, 고혈압)  Random Forest   0.796247  0.627907  0.702128   
1           (w08chronic_b, 당뇨병)  Random Forest   0.772067  0.991392  0.868090   
2      (w08chronic_c, 암 및 악성종양)  Random Forest   0.927152  1.000000  0.962199   
3        (w08chronic_d, 만성 폐질환)  Random Forest   0.970199  1.000000  0.984874   
4           (w08chronic_e, 간질환)  Random Forest   0.971302  1.000000  0.985442   
5          (w08chronic_f, 심장질환)  Random Forest   0.894040  1.000000  0.944056   
6         (w08chronic_g, 뇌혈관질환)  Random Forest   0.945856  0.998833  0.971623   
7       (w08chronic_h, 정신과적 질환)  Random Forest   0.950331  1.000000  0.974533   
8   (w08chronic_i, 관절염 또는 류마티스)  Random Forest   0.790026  0.927581  0.853296   
9       (w08chronic_k, 소화기계 질환)  Random Forest   0.985651  1.000000  0.992774   
10       (w08chronic_l, 디스크 진단)  Random Forest   0.986755  1.000000  0.993333   
11           (

In [13]:
# 전체 타겟에 대한 종합 F1 계산을 위해 모든 예측과 실제값을 한 열로 병합
all_y_test = np.concatenate([y_test[:, idx] for idx in range(y_test.shape[1])], axis=0)
all_y_pred = np.concatenate([y_pred[:, idx] for idx in range(y_pred.shape[1])], axis=0)

# 전체 타겟에 대한 평균 정확도 계산
average_accuracy = result_table['Accuracy'].mean()
print(f"\n전체 타겟에 대한 평균 정확도: {average_accuracy}")

# overall_accuracy = np.mean(np.all(y_test == y_pred, axis=1))
# print(f"\n전체 타겟에 대한 종합 정확도: {overall_accuracy}")
overall_micro_f1 = f1_score(all_y_test, all_y_pred, average='micro')
print(f"\n전체 타겟에 대한 종합 Micro F1 점수: {overall_micro_f1}")
overall_macro_f1 = f1_score(all_y_test, all_y_pred, average='macro')
print(f"\n전체 타겟에 대한 종합 Macro F1 점수: {overall_macro_f1}")
overall_weighted_f1 = f1_score(all_y_test, all_y_pred, average='weighted')
print(f"\n전체 타겟에 대한 종합 Weighted F1 점수: {overall_weighted_f1}")



전체 타겟에 대한 평균 정확도: 0.90691685062546

전체 타겟에 대한 종합 Micro F1 점수: 0.9069168506254599

전체 타겟에 대한 종합 Macro F1 점수: 0.7122863354586253

전체 타겟에 대한 종합 Weighted F1 점수: 0.8949020838649931
